In [ ]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)

## 读写文本格式的数据
Pandas提供了一些用于将表格型数据读取为DataFrame对象的函数。下表对它们进行了总结，其中read_csv和read_table可能会是你今后用得最多的。
![load_data](figures/load_data.jpg)\
我们将介绍一下这些函数在将文本数据转换为DataFrame时所用到的一些技术。这些函数的选项可以划分为以下几个大类：

* 索引：将一个或多个列当作返回的DataFrame处理，以及是否从文件中获取列名。
* 类型推断和数据转换：包括用户定义值的转换、和自定义的缺失值标记列表等。
* 日期解析：包括组合功能，比如将分散在多个列中的日期时间信息组合成结果中的单个列。
* 迭代：支持对大文件进行逐块迭代。
* 不规整数据问题：跳过一些行、页脚、注释或其他一些不重要的东西。

因为我们遇到的数据可能十分混乱，一些数据加载函数（尤其是read_csv）的选项逐渐变得复杂起来。面对不同的参数，感到头痛很正常（read_csv有超过50个参数）。pandas文档有这些参数的例子，如果你感到阅读某个文件很难，可以通过相似的例子找到正确的参数。

其中一些函数，比如pandas.read_csv，有类型推断功能，因为列数据的类型不属于数据类型。也就是说，你不需要指定列的类型到底是数值、整数、布尔值，还是字符串。其它的数据格式，如HDF5、Feather和msgpack，会在格式中存储数据类型。

首先我们来看一个以逗号分隔的（CSV）文本文件：

In [ ]:
# if you are Windows user, run this line:
!type examples\ex1.csv

In [ ]:
# if you are Mac OS or linux user, run this line
!cat examples/ex1.csv

在Shell命令前加一个感叹号，Jupyter会将其转换为Bash命令。

由于该文件以逗号分隔，所以我们可以使用read_csv将其读入一个DataFrame：

In [ ]:
df = pd.read_csv("examples/ex1.csv")
df

我们还可以使用read_table，并指定分隔符：

In [ ]:
pd.read_table('examples/ex1.csv', sep=',')

并不是所有文件都有标题行。看看下面这个文件：

In [ ]:
!cat examples/ex2.csv

读入该文件的办法有两个。你可以让pandas为其分配默认的列名，也可以自己定义列名：

In [ ]:
pd.read_csv("examples/ex2.csv", header=None)

In [ ]:
pd.read_csv("examples/ex2.csv", names=["a", "b", "c", "d", "message"])

假设你希望将message列做成DataFrame的索引。你可以通过index_col参数指定"message"：

In [ ]:
names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names, index_col="message")

如果希望将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可：

In [ ]:
!cat examples/csv_mindex.csv

In [ ]:
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

某些特殊情况下，表格可能不是用固定的分隔符去分隔字段的（比如空白符或逗号）。看看下面这个文本文件：

In [ ]:
!cat examples/ex3.txt

虽然可以手动对数据进行规整，这里的字段是被数量不同的空白字符间隔开的。这种情况下，你可以传递一个正则表达式作为read_table的分隔符。可以用正则表达式表达为\s+，于是有：

In [ ]:
result = pd.read_csv("examples/ex3.txt", sep="\s+")
result

注：在正则表达式中，\s代表空白字符，包括空格、制表符、换行符等。而+表示匹配前面的模式一次或多次。因此，\s+表示匹配一个或多个连续的空白字符。

这里，由于列名比数据行的数量少，所以read_table推断第一列应该是DataFrame的索引。

这些解析器函数还有许多参数可以帮助你处理各种各样的异形文件格式（表6-2列出了一些）。比如说，你可以用skiprows跳过文件的第一行、第三行和第四行：

In [ ]:
!cat examples/ex4.csv

In [ ]:
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

缺失值处理是文件解析任务中的一个重要组成部分。缺失数据经常是要么没有（空字符串），要么用某个标记值表示。默认情况下，pandas会用一组经常出现的标记值进行识别，比如NA及NULL：

In [ ]:
!cat examples/ex5.csv

In [ ]:
result = pd.read_csv("examples/ex5.csv")
result

In [ ]:
pd.isna(result)

na_values可以用一个列表或集合的字符串表示缺失值：

In [ ]:
result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])
result

字典的各列可以使用不同的NA标记值：

In [ ]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("examples/ex5.csv", na_values=sentinels,
            keep_default_na=False)

下表列出了pandas.read_csv和pandas.read_table常用的选项。
![pandas_read_csv](figures/pandas_read_csv.jpg)

### 逐块读取文本文件
在处理很大的文件时，或找出大文件中的参数集以便于后续处理时，你可能只想读取文件的一小部分或逐块对文件进行迭代。

在看大文件之前，我们先设置pandas显示地更紧凑些：

In [ ]:
pd.options.display.max_rows = 10

In [ ]:
result = pd.read_csv("examples/ex6.csv")
result

如果只想读取几行（避免读取整个文件），通过nrows进行指定即可：

In [ ]:
pd.read_csv("examples/ex6.csv", nrows=5)

要逐块读取文件，可以指定chunksize（行数）：

In [ ]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
type(chunker)

read_csv所返回的这个TextParser对象使你可以根据chunksize对文件进行逐块迭代。比如说，我们可以迭代处理ex6.csv，将值计数聚合到"key"列中，如下所示：

In [ ]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [ ]:
tot

### 将数据写出到文本格式
数据也可以被输出为分隔符格式的文本。我们再来看看之前读过的一个CSV文件：

In [ ]:
data = pd.read_csv("examples/ex5.csv")
data

利用DataFrame的to_csv方法，我们可以将数据写到一个以逗号分隔的文件中：

In [ ]:
data.to_csv("examples/output.csv")

In [ ]:
!cat examples/output.csv

当然，还可以使用其他分隔符（由于这里直接写出到sys.stdout，所以仅仅是打印出文本结果而已）：

In [ ]:
import sys
data.to_csv(sys.stdout, sep="|")

缺失值在输出结果中会被表示为空字符串。你可能希望将其表示为别的标记值：

In [ ]:
data.to_csv(sys.stdout, na_rep="NULL")

如果没有设置其他选项，则会写出行和列的标签。当然，它们也都可以被禁用：

In [ ]:
data.to_csv(sys.stdout, index=False, header=False)

此外，你还可以只写出一部分的列，并以你指定的顺序排列：

In [ ]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

### 处理分隔符格式
大部分存储在磁盘上的表格型数据都能用pandas.read_table进行加载。然而，有时还是需要做一些手动处理。因为读取到含有异形行的文件而使read_table出bug的情况并不少见。为了说明这些基本工具，我们看看下面这个简单的CSV文件：

In [ ]:
!cat examples/ex7.csv

对于任何单字符分隔符文件，可以直接使用Python内置的csv模块。将任意已打开的文件或文件型的对象传给csv.reader：

In [ ]:
import csv
f = open("examples/ex7.csv")
reader = csv.reader(f)

对这个reader进行迭代将会为每行产生一个元组（并移除了所有的引号）：

In [ ]:
for line in reader:
    print(line)
f.close()

现在，为了使数据格式合乎要求，你需要对其做一些整理工作。我们一步一步来做。首先，读取文件到一个多行的列表中：

In [ ]:
with open("examples/ex7.csv") as f:
    lines = list(csv.reader(f))

然后，我们将这些行分为标题行和数据行：

In [ ]:
header, values = lines[0], lines[1:]

然后，我们可以用字典构造式和zip(*values)，后者将行转置为列，创建数据列的字典：

In [ ]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

## 读取Microsoft Excel文件
pandas的ExcelFile类或pandas.read_excel函数支持读取存储在Excel 2003（或更高版本）中的表格型数据。这两个工具分别使用扩展包xlrd和openpyxl读取XLS和XLSX文件。你可以用pip或conda安装它们。

要使用ExcelFile，通过传递xls或xlsx路径创建一个实例：

In [ ]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

存储在表单中的数据可以用parse解析到DataFrame

In [ ]:
xlsx.sheet_names

In [ ]:
xlsx.parse(sheet_name="Sheet1")

In [ ]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

如果要读取一个文件中的多个表单，创建ExcelFile会更快，但你也可以将文件名传递到pandas.read_excel：

In [ ]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1")
frame

如果要将pandas数据写入为Excel格式，你必须首先创建一个ExcelWriter，然后使用pandas对象的to_excel方法将数据写入到其中：

In [ ]:
writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.close()

你还可以不使用ExcelWriter，而是传递文件的路径到to_excel：

In [ ]:
frame.to_excel("examples/ex2.xlsx")

In [ ]:
!rm examples/ex2.xlsx

### This is the end! :)